<a href="https://colab.research.google.com/github/b62mp1/ExistingResults/blob/main/Copy_of_Lstm_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 10.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go


In [ ]:
stock_name = "GOOG"
dataset=pdr.get_data_yahoo(stock_name, start='2015-01-01', end='2018-03-08').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (800, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-02,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447563
2015-01-05,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059840
2015-01-06,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899940
2015-01-07,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065054
2015-01-08,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353582


In [ ]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

'''
scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)'''

'\nscaler = MinMaxScaler(feature_range=(0, 1))\nclose_data = scaler.fit_transform(close_data)'

In [ ]:

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]


In [ ]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
model = Sequential()
model.add(
    LSTM(1000,
        activation='relu',
        input_shape=(look_back,1)
        )
)

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/25
32/32 [==============================] - 19s 549ms/step - loss: 90994.1953
Epoch 2/25
32/32 [==============================] - 22s 682ms/step - loss: 34403.2461
Epoch 3/25
32/32 [==============================] - 12s 390ms/step - loss: 1821.5052
Epoch 4/25
32/32 [==============================] - 13s 393ms/step - loss: 1946.5640
Epoch 5/25
32/32 [==============================] - 13s 392ms/step - loss: 2239.7820
Epoch 6/25
32/32 [==============================] - 12s 388ms/step - loss: 2807.0244
Epoch 7/25
32/32 [==============================] - 12s 387ms/step - loss: 6104.5923
Epoch 8/25
32/32 [==============================] - 12s 390ms/step - loss: 2623.3303
Epoch 9/25
32/32 [==============================] - 13s 413ms/step - loss: 2503.8477
Epoch 10/25
32/32 [==============================] - 13s 392ms/step - loss: 4024.2007
Epoch 11/25
32/32 [==============================] - 12s 389ms/step - loss: 1374.5165
Epoch 12/25
32/32 [==============================] - 13s 394m

In [ ]:
prediction = model.predict(test_generator)

In [ ]:

def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [ ]:
#close_test=close_test[:-look_back]

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)


trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode='lines',
    name = 'Future Data'
)
layout = go.Layout(
    title = stock_name,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [ ]:
'''close_test2=close_test[:-look_back]
RNNmse = mean_squared_error(close_test2,prediction)
from math import sqrt
RNNrmse = sqrt(RNNmse)
RNNr2Score = r2_score(close_test2,prediction)

RNNmse,RNNrmse,RNNr2Score'''

(1155.2916503266786, 33.989581496786315, 0.7846391205360544)